In [1]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime

Importing plotly failed. Interactive plots will not work.


# Global Model

In [2]:
no_of_records = 4677196

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'air_quality_pmaqi'
pipeline = [
    {
        "$sample": {"size": no_of_records // 100}
    }
]
cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [3]:
def format_date(t):
    t = str(t)
    date = datetime.strptime(t, '%Y%m%d%H')
    return f'{date.year}-{date.month}-{date.day} {str(date.hour).zfill(2)}:{str(date.minute).zfill(2)}:{str(date.second).zfill(2)}'


df = df_original
df['gmtTimestampBegin'] = df['gmtTimestampBegin'].apply(format_date)
df

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,609099e36e34e1996d45e6f6,2010-1-11 00:00:00,0,G4600310,45.849128,-101.661167,102420.0,93993.0,690.975159,268.547516,...,-2.606369,0.000,0.00,266.066574,20360.487366,24036.791128,9.101855,100.0,0.11800,0.0
1,60909a989537f426fbb566c0,2010-1-12 12:00:00,6,G4100630,45.829541,-117.133598,101186.0,86877.0,1248.725159,274.036041,...,1.906586,0.000,0.00,272.285583,24560.490417,24037.305707,12.518266,100.0,0.02296,0.0
2,60909c8f5a88413eaaa370df,2010-1-17 06:00:00,3,G0500550,36.129356,-90.264987,100774.0,99836.0,78.725159,278.360413,...,-6.903336,5.000,0.00,277.735031,22160.510254,10037.354654,26.555961,100.0,0.00000,0.0
3,60909b4e184c9c5cb32e928d,2010-1-14 06:00:00,3,G2300030,46.737189,-69.329707,102554.0,97934.0,342.475159,246.688293,...,1.086803,0.000,0.00,267.472977,22160.438538,4633.232546,3.318760,100.0,0.33896,0.0
4,60909da4c821587ed708d140,2010-1-19 18:00:00,3,G4100450,44.209585,-117.635051,99662.0,85271.0,1278.725159,276.965988,...,0.828339,0.000,0.00,273.036484,28360.485840,24234.204668,6.298174,2.0,0.00784,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74310,60909aa3825266f81ff65a18,2010-1-12 18:00:00,3,G1300210,32.880804,-83.661663,102362.0,100792.0,128.225159,280.421005,...,-2.711548,0.000,0.00,273.506485,11560.510254,24229.832041,6.587578,0.0,0.00000,0.0
74311,60909d41cff87b5fe03721fd,2010-1-18 18:00:00,6,G3000170,46.072444,-105.797155,100719.0,90573.0,861.725159,269.217453,...,1.465881,0.000,0.00,270.460312,22160.461426,24233.817530,8.176936,100.0,0.02840,0.0
74312,60909b981b8923fd3ec956ea,2010-1-15 00:00:00,3,G4800490,31.593235,-99.151984,101811.0,96559.0,445.975159,283.434509,...,0.652668,0.125,0.00,284.061935,26173.812866,231.664586,18.644155,100.0,0.00000,0.0
74313,60909c21a4390eb9da0b2f73,2010-1-16 06:00:00,3,G0100550,34.123737,-86.159231,102262.0,98859.0,276.475159,273.396027,...,0.308901,0.000,0.00,275.026611,18374.395752,24037.214273,11.763236,100.0,0.00000,0.0


In [10]:
df = df_original
unique_timestamps = df['gmtTimestampBegin'].unique()
print(f'#timestamps: {len(unique_timestamps)}')

selected_feature = 'aqi'
gis_join = 'gisJoin'

df_s = df[[gis_join, 'gmtTimestampBegin', selected_feature]]
means = []
for t in unique_timestamps:
    mean = df_s[df_s['gmtTimestampBegin'] == t][selected_feature].mean()
    means.append(mean)
df_aqi = pd.DataFrame(list(zip(list(df_s[gis_join]), unique_timestamps, means)), columns=[gis_join, 'ds', 'y'])
# df_means.index.name = 'ds'

#timestamps: 8166


In [12]:
df_aqi.head()

,gisJoin,ds,y
0,G2200190002700,2019-03-29,42.503293
1,G0600650040607,2010-10-31,42.804951
2,G4200030521100,2012-10-29,28.238995
3,G0600670001700,1999-01-22,87.996673
4,G3600810049302,2018-08-14,48.572129


In [14]:
pickle.dump(df_aqi, open('pickles/aqi/df_aqi.pkl', 'wb'))

In [3]:
# Start by loading pickled 'df_map'
df_aqi = pickle.load(open('pickles/aqi/df_aqi.pkl', 'rb'))

## Build Global Model

In [4]:
def predict(df_train):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return fit_m, df_train_future, df_train_forecast

fit_m, df_train, df_train_forecast = predict(df_aqi)

Initial log joint probability = -245.244
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       14802.1   0.000113044       91.9588      0.7255      0.7255      142   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       14805.1   3.87898e-05       48.2919      0.4863      0.4863      272   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     236       14805.2   4.17365e-05       71.8587   2.058e-07       0.001      373  LS failed, Hessian reset 
     299       14805.5    0.00440374       109.649           1           1      449   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     311       14805.6   5.36995e-05        121.06    6.94e-07       0.001      516  LS failed, Hessian reset 
     399       14805.8   3.27779e-05       69.6101           1           1      630   
    Iter      log pro

In [5]:
df_train_forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1989-01-22,159.741302,63.340348,116.806454,159.741302,159.741302,-69.438756,-69.438756,-69.438756,-62.680051,...,-3.506350,-3.506350,-3.506350,-3.252355,-3.252355,-3.252355,0.0,0.0,0.0,90.302545
1,1989-10-07,158.447869,63.792868,118.555154,158.447869,158.447869,-65.005589,-65.005589,-65.005589,-62.680051,...,-1.854341,-1.854341,-1.854341,-0.471197,-0.471197,-0.471197,0.0,0.0,0.0,93.442280
2,1990-04-02,157.560514,63.414645,116.881752,157.560514,157.560514,-68.087624,-68.087624,-68.087624,-62.680051,...,-0.062498,-0.062498,-0.062498,-5.345075,-5.345075,-5.345075,0.0,0.0,0.0,89.472891
3,1990-04-16,157.490328,63.211394,118.461607,157.490328,157.490328,-66.882286,-66.882286,-66.882286,-62.680051,...,-0.062498,-0.062498,-0.062498,-4.139737,-4.139737,-4.139737,0.0,0.0,0.0,90.608042
4,1990-08-05,156.933851,70.686548,124.977854,156.933851,156.933851,-58.325953,-58.325953,-58.325953,-62.680051,...,-3.506350,-3.506350,-3.506350,7.860447,7.860447,7.860447,0.0,0.0,0.0,98.607898


In [6]:
global_model = fit_m

In [7]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df0_cv = cross_validation(fit_m, initial='2 days', period='1 days', horizon='1 days')
df0_p = performance_metrics(df0_cv)
df0_p.head()

KeyboardInterrupt: 

In [8]:
model = fit_m
print(f'seasonality_prior_scale: {model.seasonality_prior_scale}')
print(f'changepoint_prior_scale: {model.changepoint_prior_scale}')
print(f'uncertainty_samples: {model.uncertainty_samples}')
print(f'seasonlity_mode: {model.seasonality_mode}')
print(f'interval_width: {model.interval_width}')
print(f'growth: {model.growth}')

seasonality_prior_scale: 10.0
changepoint_prior_scale: 0.05
uncertainty_samples: 1000
seasonlity_mode: additive
interval_width: 0.8
growth: linear


## GridSearch on Global Model

In [20]:
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')    


def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '1000 days', period='5 days', horizon='5 days', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale  

In [21]:
rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df_aqi)

# Optimal values
print(f'rmse: {rmse}')
print(f'changepoint_prior_scale: {changepoint_prior_scale}')
print(f'seasonality_prior_scale: {seasonality_prior_scale}')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -245.244
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      51       14644.8    0.00352726       4410.57   8.078e-07       0.001      104  LS failed, Hessian reset 
      99       14650.6   1.34709e-06       4141.06      0.7048      0.7048      173   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     144       14650.7   1.58378e-05       3583.63   4.132e-09       0.001      266  LS failed, Hessian reset 
     172       14650.8   1.80777e-08       2522.84           1           1      308   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


INFO:prophet:Making 1974 forecasts with cutoffs between 1991-12-08 00:00:00 and 2019-12-27 00:00:00
INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


KeyboardInterrupt: 

In [24]:
gis_joins = df_original['gisJoin'].unique()
print(f'#GISJOINs: {len(gis_joins)}')

#GISJOINs: 1462


## Dictionary of GISJOIN --> DF

In [25]:
import dask
selected_feature = 'aqi'
child_dfs = {}

def get_df_by_gis_join(gis_join_):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"gisJoin": gis_join_}}])
    df = pd.DataFrame(list(cursor))[['gmtTimestampBegin', selected_feature]]
    df.columns = ['ds','y']
    return df

futures = []
for gis_join in gis_joins:
    child_dfs[gis_join] = get_df_by_gis_join(gis_join)

G1200310016000 G2700750370100 G3400130023200 G4801410003602 G5100590420800 G2800330071120 G2800810980000 G0600730019101 G5300610053700 G1901770950200 G3001110000402 G4601030011400 G2601610410800 G5300530060200 G1302450010508 G3300090961800 G1200110070602 G0400010944000 G0800010015000 G3200030003253 G4600990000500 G4200490000600 G4700370012100 G4201250783200 G4804530001765 G3000870940400 G0600270000800 G3600610020901 G5300110040102 G0600790012502 G1200470960300 G4801490970700 G4800290141900 G2500050640200 G1200950015901 G0600650031401 G4801350001700 G2200730980000 G5300610051000 G2900470021805 G4600330965100 G3000810000201 G1901030001200 G0900030502100 G0100730010900 G0600110000200 G5400390001300 G0100730011206 G0600870120301 G3000530000200 G2700050450400 G2601630524700 G1500010020802 G0600370206010 G5600210001402 G1200850001000 G1701570950700 G3500130001804 G1200570013604 G4100190080000 G0100730005500 G1000050050406 G2500030900100 G3700570061500 G0601110006600 G0600250010102 G420003003

G0200200000202 G0600790011103 G3000490000400 G4701730040100 G4800430950500 G1100010009604 G5600050000100 G3400390035900 G5600130000300 G4001130940002 G3300050971100 G4200030562900 G4802030020102 G1200050002607 G1600690960600 G2300090965900 G3800530962300 G4802450006600 G5500270961400 G2200330005100 G4201010980300 G4900050001002 G1200990008101 G1600050940000 G2001810453700 G2700030050813 G4000010376800 G4800610014001 G0400130319704 G2200510020517 G1201030027310 G2200710005604 G2500090260900 G4200770009600 G0600270000500 G2900770004400 G4400070000400 G0900050263200 G2500230511301 G5300730000803 G0400150951402 G1800950001300 G5101610031102 G5501330202800 G3700630000102 G0600130368002 G5600230978200 G2905100127000 G3700650020700 G0600650043006 G0600770005122 G2701230034202 G5300330032319 G1000030002700 G4200030060300 G4400030020800 G1300210013900 G0100730014203 G0600310001100 G0900030510600 G4200790201600 G3200210970800 G4701570980400 G5600350000102 G4400070000800 G0400010970502 G270083036

G4801390060802 G0600670008404 G0600090000300 G4100290002700 G0400130941300 G2700530100500 G4100050024304 G1000030014810 G1200890050202 G2700210940002 G1201150001301 G0400190002501 G4200270011600 G0400190002300 G0400070001000 G0400130941200 G4803030010200 G4200750003200 G3800070963100 G0900090142700 G3701230960500 G1200570002200 G0600730021100 G1701190400102 G0800410004901 G0600790012402 G4800290121203 G2200790013300 G4100190200000 G4500250950800 G1200010002101 G0601110000902 G2900990700110 G1600830000700 G1200010001803 G0600830002606 G3600610001501 G5300530062000 G5600370971600 G5500630001102 G2101110009400 G4700990960700 G1200990005501 G0800310000202 G0501390950700 G5600350000101 G3701050030602 G3500010003731 G0600630000100 G3000310000701 G0400210941300 G4701250101305 G1600850970300 G1301150001400 G4001050172200 G3500450000504 G3200030001612 G0800930000400 G0200900000100 G3200310003501 G5100590431600 G1801270050405 G1702010001000 G2102270011600 G4201150032500 G5300230970300 G410071003

In [27]:
# pickle.dump(child_dfs, open('pickles/aqi/child_dfs.pkl', 'wb'))
child_dfs = pickle.load(open('pickles/aqi/child_dfs.pkl', 'rb'))

In [38]:
class TLModel:    
    def __init__(self, gis_join, time, model):
        self.gis_join = gis_join
        self.time = time
        self.model = model
        
    def __str__(self):
        return f'{self.gis_join}: TLModel(time={self.time})'
    
class NonTLModel:
    def __init__(self, gis_join, time):
        self.gis_join = gis_join
        self.time = time
        
    def __str__(self):
        return f'{self.gis_join}: NonTLModel(time={self.time})'

# Child Models (Non-TL)

In [36]:
import time

non_tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:500]:
    time1 = time.monotonic()
    predict(child_dfs[gis_join])
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    non_tl_models[gis_join] = NonTLModel(gis_join, time2)
    counter += 1

Initial log joint probability = -8.10668
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3773.38    0.00233689       95.8493       0.232           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3777.77   0.000349536       220.986      0.5748      0.5748      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3788.52   0.000631608       383.067      0.5601      0.5601      363   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3791.47   0.000427022        98.791      0.4315      0.4315      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     487       3797.76   0.000124837       162.669   3.931e-07       0.001      647  LS failed, Hessian reset 
     499       3799.34    0.00312868    

5 - G5100590420800: 5.450836025876924 seconds
Initial log joint probability = -100.833
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7168.03   6.57676e-05       72.7244      0.8933      0.8933      143   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7168.15   5.57092e-06       69.7958           1           1      277   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     222       7168.15   3.28766e-07       55.4687      0.2818           1      310   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
6 - G2800330071120: 5.874652112834156 seconds
Initial log joint probability = -35.2651
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       5628.38   0.000836687       67.1427           1           1      130   
    

11 - G3001110000402: 5.352875056909397 seconds
Initial log joint probability = -45.32
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       14071.2   0.000446591       399.202      0.4602      0.4602      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     134       14072.5   0.000106576       267.599   1.655e-07       0.001      215  LS failed, Hessian reset 
     199       14073.4   0.000630468        490.59           1           1      304   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     253       14074.1   6.24848e-05       181.521   1.802e-07       0.001      408  LS failed, Hessian reset 
     299       14074.4   0.000563424       72.8895           1           1      465   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       14074.9   9.44853e-05       80.9227     0.04232 

17 - G1200110070602: 4.242634230060503 seconds
Initial log joint probability = -121.749
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3035.25    0.00259223       79.4217           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     160       3035.93   0.000491486       103.363   6.534e-06       0.001      244  LS failed, Hessian reset 
     199       3036.21   0.000101151       78.1193      0.7658      0.7658      287   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     244        3036.5   0.000500558       85.1719   5.687e-06       0.001      400  LS failed, Hessian reset 
     299       3036.69   4.90002e-05       98.8171      0.7399      0.7399      465   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     359       3036.75   2.18447e-06       80.6143   2.579e-0

23 - G4700370012100: 9.289975464111194 seconds
Initial log joint probability = -269.747
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       12215.6    0.00608002       262.714           1           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     145       12222.1   0.000215063       302.421   1.007e-06       0.001      220  LS failed, Hessian reset 
     163       12224.7   0.000692909       324.998   1.007e-05       0.001      276  LS failed, Hessian reset 
     199       12226.5    0.00209255       132.284           1           1      317   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     245       12229.4   0.000753272       261.741   1.076e-05       0.001      425  LS failed, Hessian reset 
     299       12231.1   0.000539862       66.2141      0.3113           1      495   
    Iter      log prob        ||dx||      

...

469 - G5300270001200: 12.998651029076427 seconds
Initial log joint probability = -13.231
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2787.72   0.000199698       233.592      0.1893      0.1893      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       2788.39    0.00136437       450.883   1.261e-05       0.001      189  LS failed, Hessian reset 
     199       2788.56   0.000219143       91.0572       0.809      0.1947      291   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     226       2788.93   0.000113073       88.3802   7.113e-07       0.001      375  LS failed, Hessian reset 
     299       2789.19   8.58816e-06       66.7687           1           1      469   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     327       2789.19   8.83943e-06       54.6562   1.259e-

478 - G2300050001500: 6.5609021950513124 seconds
Initial log joint probability = -106.822
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       12460.4    0.00224027       118.723           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       12464.2    0.00157036       224.699           1           1      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     296       12465.4   0.000332126       299.602   3.154e-06       0.001      406  LS failed, Hessian reset 
     299       12465.4   0.000241776       109.851           1           1      409   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       12466.2   8.94558e-05        78.327      0.6476      0.6476      540   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals

485 - G4801410001502: 17.544837145134807 seconds
Initial log joint probability = -88.6976
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4900.43     0.0105286        298.51       1.763      0.1763      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     160        4905.1   0.000280297       164.716   2.583e-06       0.001      242  LS failed, Hessian reset 
     199       4906.09    0.00108415       61.7411           1           1      291   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     217       4906.46   0.000120824       81.2183   1.306e-06       0.001      358  LS failed, Hessian reset 
     277       4907.52   0.000177908       107.343   1.188e-06       0.001      486  LS failed, Hessian reset 
     299       4907.61   9.28553e-05        72.443      0.7924      0.7924      516   
    Iter      log prob        ||dx||    

490 - G4801410010409: 9.282641015946865 seconds
Initial log joint probability = -29.7921
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2483.67    0.00115729       81.9296      0.4161      0.4161      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2486.59   0.000400972       84.9169           1           1      253   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     228       2487.13   0.000683534       175.399   5.804e-06       0.001      334  LS failed, Hessian reset 
     282       2487.62   6.25469e-05       68.6165   7.563e-07       0.001      438  LS failed, Hessian reset 
     299       2487.66   1.29915e-05       60.1966      0.2687      0.2687      459   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     393       2487.67    3.1075e-07       70.0371      0.40

Initial log joint probability = -81.9419
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       6141.97    0.00191936       119.624      0.7784      0.7784      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     132       6146.05   0.000202668       116.435   8.481e-07       0.001      201  LS failed, Hessian reset 
     199       6153.27    0.00514444       114.281           1           1      288   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     240       6155.69   0.000193846        96.851   8.033e-07       0.001      374  LS failed, Hessian reset 
     296        6157.8   0.000457925       297.891   1.487e-06       0.001      490  LS failed, Hessian reset 
     299       6158.12    0.00282582       157.835       9.963      0.9963      494   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

In [29]:
import time
import dask
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')

def predict_parallel(df_train, gis_join):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300)
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return gis_join, (time2 - time1)


gis_joins_list = []
child_dfs_list = []
# cutoff = 3000

for gis_join, df_ in child_dfs.items():
    gis_joins_list.append(gis_join)
    child_dfs_list.append(df_)


def train_one_child(df_, gis_join):
    gis_join, time_taken = predict_parallel(df_, gis_join)
    return gis_join, time_taken

assert len(gis_joins_list) == len(child_dfs_list)

client.scatter(child_dfs_list)

counter = 1
lazy_results = []
for gis_join, df_ in zip(gis_joins_list, child_dfs_list):
    lazy_result = dask.delayed(train_one_child)(df_, gis_join)
    lazy_results.append(lazy_result)
    print(f'{counter}')
    counter += 1

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


(('G1200310016000', 65.12528231088072),
 ('G2700750370100', 105.10045588295907),
 ('G3400130023200', 118.05747111397795),
 ('G4801410003602', 122.90893771313131),
 ('G5100590420800', 107.43155716708861))

In [30]:
print(f'#results: {len(results)}')
results[100]

#results: 1462


('G3000490000300', 84.57192057580687)

# Child Models (TL)

In [39]:
def predict_transfer(df_train, trained_model):
    m_parent = trained_model
    # initilaize model with hyperparameters from parent model
    m = Prophet(
        seasonality_prior_scale = m_parent.seasonality_prior_scale,
        changepoint_prior_scale = m_parent.changepoint_prior_scale,
    )
    m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return m, df_train_future, df_train_forecast

tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:500]:
    time1 = time.monotonic()
    m, df_train_future, df_train_forecast = predict_transfer(child_dfs[gis_join], global_model)
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    tl_models[gis_join] = TLModel(gis_join, time2, m)
    counter += 1 

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -8.10668
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3776.23   0.000927946       128.834           1           1      115   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     162       3782.93   0.000214436       155.312   1.036e-06       0.001      242  LS failed, Hessian reset 
     199       3789.05    0.00299258       150.578      0.9961      0.9961      289   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3793.03    0.00164627       99.3114       0.794       0.794      415   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     378       3799.06   0.000467629       249.731   2.529e-06       0.001      559  LS failed, Hessian reset 
     399       3800.13   0.000427223       224.992      0.5102      0.5102      586   
    Iter      log pro

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 - G1200310016000: 6.142463716911152 seconds
Initial log joint probability = -46.949
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       8889.53    0.00383671       331.137      0.2248      0.8648      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       8895.65    0.00375264       145.228      0.9485      0.9485      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     298       8897.81   0.000303157       111.886   4.666e-06       0.001      421  LS failed, Hessian reset 
     299       8897.81   0.000348423       78.6348           1           1      422   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       8898.01   7.47514e-05       63.5656       0.602       0.602      544   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  No

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

497 - G4200450410700: 8.51229047193192 seconds
Initial log joint probability = -72.7203
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4313.51   1.27537e-05       56.6962       0.125       0.347      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4313.59   0.000594031       68.8903           1           1      253   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       4313.86   0.000566341       82.1742           1           1      379   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     331       4313.88   2.17098e-07       64.8834      0.3771           1      419   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


498 - G2300190000200: 5.423748805187643 seconds
Initial log joint probability = -22.9129
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       8205.23    0.00673547       144.784           1           1      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       8209.28   0.000100139        73.827      0.4528      0.4528      250   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     254       8209.81   8.12276e-05       82.8592   4.436e-07       0.001      381  LS failed, Hessian reset 
     299       8210.19   4.61371e-06       71.6288           1           1      441   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     308        8210.2   5.07948e-05       86.4467   6.779e-07       0.001      501  LS failed, Hessian reset 
     344        8210.2   3.33349e-06       72.4766   5.055e-

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


499 - G0600310001500: 6.3127833949401975 seconds
Initial log joint probability = -15.8149
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       6122.81   0.000145315       74.9266      0.5027      0.5027      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     132       6122.98   0.000369304       88.2225   4.656e-06       0.001      209  LS failed, Hessian reset 
     199       6123.15   0.000128669       122.354   1.495e-06       0.001      349  LS failed, Hessian reset 
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     296       6123.19   2.25261e-07       87.9806      0.2324       0.956      474   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
500 - G0400210941400: 5.617289903108031 seconds


In [31]:
import dask
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')

def predict_transfer_parallel(df_train, gis_join):
    # initilaize model with hyperparameters from parent model
    time1 = time.monotonic()
    m = Prophet(
        seasonality_prior_scale = 1.0,
        changepoint_prior_scale = 0.001,
    )
    m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    
    time2 = time.monotonic()

    return gis_join, (time2 - time1)

def train_one_child(df_, gis_join):
    gis_join, time_taken = predict_transfer_parallel(df_, gis_join)
    return gis_join, time_taken

In [32]:
assert len(gis_joins_list) == len(child_dfs_list)

client.scatter(child_dfs_list)

counter = 1
lazy_results = []
for gis_join, df_ in zip(gis_joins_list, child_dfs_list):
    lazy_result = dask.delayed(train_one_child)(df_, gis_join)
    lazy_results.append(lazy_result)
    print(f'{counter}')
    counter += 1

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


(('G1200310016000', 53.52447824901901),
 ('G2700750370100', 75.31010662904009),
 ('G3400130023200', 47.164661552989855),
 ('G4801410003602', 97.859520789003),
 ('G5100590420800', 97.93322589388117))

In [33]:
print(f'#results: {len(results)}')
results[100]

#results: 1462


('G3000490000300', 91.09751586895436)

## Collect Times taken to build Models

In [40]:
import pandas as pd

non_tl_times = [x.time for x in non_tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(non_tl_models.keys(), non_tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('aqi_non_tl_times.csv', index=False)

In [41]:
tl_times = [x.time for x in tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(tl_models.keys(), tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('aqi_tl_times.csv', index=False)